In [1]:
import pandas as pd

In [2]:
trips_df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'client_id': [1, 2, 3, 4, 1, 2, 3, 2, 3, 4],
    'driver_id': [10, 11, 12, 13, 10, 11, 12, 12, 10, 13],
    'city_id': [1, 1, 6, 6, 1, 6, 6, 12, 12, 12],
    'status': ['completed', 'cancelled_by_driver', 'completed', 'cancelled_by_client', 'completed', 'completed', 'completed', 'completed', 'completed', 'cancelled_by_driver'],
    'request_at': ['2013-10-01', '2013-10-01', '2013-10-01', '2013-10-01', '2013-10-02', '2013-10-02', '2013-10-02', '2013-10-03', '2013-10-03', '2013-10-03']
})
trips_df

,id,client_id,driver_id,city_id,status,request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [3]:
users_df = pd.DataFrame({
    'users_id': [1, 2, 3, 4, 10, 11, 12, 13],
    'banned': [False, True, False, False, False, False, False, False],
    'role': ['client', 'client', 'client', 'client', 'driver', 'driver', 'driver', 'driver']
})
users_df

,users_id,banned,role
0,1,False,client
1,2,True,client
2,3,False,client
3,4,False,client
4,10,False,driver
5,11,False,driver
6,12,False,driver
7,13,False,driver


How to calculate the cancellation rate of requests with unbanned users (client and driver both not banned) each day between October 1, 2013, and October 3, 2013? Ensure the solution rounds the cancellation rate to two decimal points.

In [4]:
output = pd.DataFrame({
    "Day": ["2013-10-01", "2013-10-02", "2013-10-03"],
    "Cancellation Rate": [0.33, 0.00, 0.50]
})
output

,Day,Cancellation Rate
0,2013-10-01,0.33
1,2013-10-02,0.00
2,2013-10-03,0.50


Explanation:

On 2013-10-01: Out of 4 total requests, 2 were canceled. Discounting a request from a banned client, there were 3 unbanned requests, with a cancellation rate of 0.33.

On 2013-10-02: Among 3 total requests, none were canceled. Ignoring a request from a banned client, there were 2 unbanned requests, resulting in a cancellation rate of 0.00.

On 2013-10-03: Out of 3 total requests, 1 was canceled. Disregarding a request from a banned client, there were 2 unbanned requests, leading to a cancellation rate of 0.50.

# Merging two Dataframes

In [5]:
merge_by_client_id = pd.merge(trips_df, users_df, left_on='client_id', right_on='users_id')
merge_by_client_id

,id,client_id,driver_id,city_id,status,request_at,users_id,banned,role
0,1,1,10,1,completed,2013-10-01,1,False,client
1,5,1,10,1,completed,2013-10-02,1,False,client
2,2,2,11,1,cancelled_by_driver,2013-10-01,2,True,client
3,6,2,11,6,completed,2013-10-02,2,True,client
4,8,2,12,12,completed,2013-10-03,2,True,client
5,3,3,12,6,completed,2013-10-01,3,False,client
6,7,3,12,6,completed,2013-10-02,3,False,client
7,9,3,10,12,completed,2013-10-03,3,False,client
8,4,4,13,6,cancelled_by_client,2013-10-01,4,False,client
9,10,4,13,12,cancelled_by_driver,2013-10-03,4,False,client


In [6]:
total_df = pd.merge(merge_by_client_id, users_df,
                           left_on='driver_id',
                           right_on='users_id', 
                           suffixes=('_client', '_driver'))
total_df

,id,client_id,driver_id,city_id,status,request_at,users_id_client,banned_client,role_client,users_id_driver,banned_driver,role_driver
0,1,1,10,1,completed,2013-10-01,1,False,client,10,False,driver
1,5,1,10,1,completed,2013-10-02,1,False,client,10,False,driver
2,9,3,10,12,completed,2013-10-03,3,False,client,10,False,driver
3,2,2,11,1,cancelled_by_driver,2013-10-01,2,True,client,11,False,driver
4,6,2,11,6,completed,2013-10-02,2,True,client,11,False,driver
5,8,2,12,12,completed,2013-10-03,2,True,client,12,False,driver
6,3,3,12,6,completed,2013-10-01,3,False,client,12,False,driver
7,7,3,12,6,completed,2013-10-02,3,False,client,12,False,driver
8,4,4,13,6,cancelled_by_client,2013-10-01,4,False,client,13,False,driver
9,10,4,13,12,cancelled_by_driver,2013-10-03,4,False,client,13,False,driver


# Getting the Cancelation Rate

In [7]:
total_requests_per_day = trips_df.groupby('request_at').size()
total_requests_per_day.to_frame(name='total_requests_per_day')

,total_requests_per_day
request_at,
2013-10-01,4
2013-10-02,3
2013-10-03,3


In [8]:
cancelled_requests_per_day = trips_df[trips_df['status'].str.contains(
    'cancelled')].groupby('request_at').size()

cancelled_requests_per_day.to_frame(name='cancelled_requests_per_day')

,cancelled_requests_per_day
request_at,
2013-10-01,2
2013-10-03,1


In [9]:
unbanned_requests = total_df[(total_df['banned_client'] == False) 
                                 & (total_df['banned_driver'] == False)]
unbanned_requests

,id,client_id,driver_id,city_id,status,request_at,users_id_client,banned_client,role_client,users_id_driver,banned_driver,role_driver
0,1,1,10,1,completed,2013-10-01,1,False,client,10,False,driver
1,5,1,10,1,completed,2013-10-02,1,False,client,10,False,driver
2,9,3,10,12,completed,2013-10-03,3,False,client,10,False,driver
6,3,3,12,6,completed,2013-10-01,3,False,client,12,False,driver
7,7,3,12,6,completed,2013-10-02,3,False,client,12,False,driver
8,4,4,13,6,cancelled_by_client,2013-10-01,4,False,client,13,False,driver
9,10,4,13,12,cancelled_by_driver,2013-10-03,4,False,client,13,False,driver


In [10]:
unbanned_requests_per_day = total_df[(total_df['banned_client'] == False)
                                     & (total_df['banned_driver'] == False)].groupby('request_at').size()
unbanned_requests_per_day.to_frame(name='unbanned_requests_per_day')

,unbanned_requests_per_day
request_at,
2013-10-01,3
2013-10-02,2
2013-10-03,2


In [11]:
unbanned_cancelled_requests_per_day = total_df[(total_df['banned_client'] == False) 
                                               & (total_df['banned_driver'] == False) 
                                               & (total_df['status'].str.contains('cancelled'))].groupby('request_at').size()

unbanned_cancelled_requests_per_day.to_frame(name='unbanned_cancelled_requests_per_day')

,unbanned_cancelled_requests_per_day
request_at,
2013-10-01,1
2013-10-03,1


In [12]:
cancellation_rates_per_day = (unbanned_cancelled_requests_per_day
                              / unbanned_requests_per_day * 100).fillna(0).round(2)
cancellation_rates_per_day.to_frame(name='cancellation_rates_per_day')

,cancellation_rates_per_day
request_at,
2013-10-01,33.33
2013-10-02,0.00
2013-10-03,50.00
